In [3]:
!pip install transliterate

  Using cached transliterate-1.10.2-py2.py3-none-any.whl (45 kB)


In [5]:
!pip install pymorphy2

  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
Processing c:\users\vital\appdata\local\pip\cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e\docopt-0.6.2-py2.py3-none-any.whl
  Using cached pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl (7.1 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)


In [6]:
import transliterate
from transliterate import translit, get_available_language_codes
import pymorphy2
import os
import re
import string
from collections import Counter
from nltk.corpus import stopwords

# Тося

## Скачивание файлов

In [7]:
folder = []
for i in os.walk('childes_Tosia'):
    folder.append(i)

Выводим список всех файлов и папок 

In [3]:
#folder

In [8]:
filenames = []
for address, dirs, files in folder:
    for file in files:
        filenames.append(file)

Выводим список всех файлов, содержащих диалоги

In [5]:
#filenames

In [9]:
paths = []  # список готовых директорий к файлам с диалогами
for filename in filenames:
    if filename.startswith('TD'):
        directory = 'childes_Tosia/malenkaya/'
    else:
        directory = 'childes_Tosia/'
    path = directory + filename
    paths.append(path)
    #print(path)

#print(len(paths))

## Разделение реплик на то, что ребенок слышит, и то, что он говорит

In [10]:
morph = pymorphy2.MorphAnalyzer()

dict_words = []

textChild = ''
textParents = ''

for filename in paths:
    with open(filename, 'r', encoding='utf-8') as f:
        file = f.readlines()
        for line in file:
            line = translit(line, 'ru')
            line = re.sub('йо', 'ё', line)
            if line.startswith('*ЦХИ'):  # выбираем только реплики ребенка 
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textChild += line.lower()
            elif line.startswith('*'):
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textParents += line.lower()
        
with open('TosiaSays.txt', 'w+', encoding='utf-8') as f:
    f.write(textChild)
with open('TosiaHears.txt', 'w+', encoding='utf-8') as f:
    f.write(textParents)

In [50]:
def WordsCounting(filepath):
    with open(filepath, 'r+', encoding='utf-8') as f:
        text = f.readlines()
        # список из всех слов, которые говорит или слышит ребенок
        words = []
        for line in text:
            if ' [: ' in line:
                line = line.replace(' [:', 'ERR')
            for word in line.lower().translate(str.maketrans(dict.fromkeys(string.punctuation))).split():
                if not word.endswith('err'):
                    words.append(word)

    # список из словрей вида слово:часть речи
    dict_words = [] 
    for word in words:
        pos = morph.parse(word)[0].tag.POS
        d = {}
        d[word] = pos
        dict_words.append(d)
    
    # список из частей речи всех слов -- для частеречного анализа
    pos = []
    len_corpus = 0
    for d in dict_words:
        for values in d.values():
            len_corpus += 1
            if values != None:
                pos.append(values)
    
    # топ частей речи, которые слышит или говорит ребенок
    top_pos = Counter(pos)
    print('Распределение по частям речи ', top_pos)
    
    # общая размерность корпуса вместе со словами, у которых тэг части речи None
    print('Всего слов в этой части коруса (вместе с неопределенной частью речи)', len_corpus)
    
    # топ-10 слов и топ-10 существительных, которые слышит или говорит ребенок
    all_words = []
    all_nouns = []
    forbidden = ['цхи', 'ххх', 'мот', 'фат', 'æто',
                 'лаугхинга', 'чико', 'нана', 'мотхер', 'бро',
                 'мота', 'xxx', 'сис']
    for d in dict_words:
        for key, value in d.items():
            key = morph.parse(key)[0].normal_form
            if key not in forbidden:
                all_words.append(key)
            if value == 'NOUN' and key not in forbidden:
                all_nouns.append(key)
                
    sw = stopwords.words('russian')
    
    filtered_words = [w for w in all_words if w not in sw]
    filtered_nouns = [w for w in all_nouns if w not in sw]
    
    top_words = Counter(filtered_words)
    top_words = dict(top_words)
    top_words = {c: top_words[c] for c in top_words if top_words[c] > 2 and len(c) > 2}
    top_words = (sorted(top_words.items(), key=lambda x: x[1], reverse=True))[:10]
    
    top_nouns = Counter(filtered_nouns)
    top_nouns = dict(top_nouns)
    top_nouns = {c: top_nouns[c] for c in top_nouns if top_nouns[c] > 2 and len(c) > 2}
    top_nouns = (sorted(top_nouns.items(), key=lambda x: x[1], reverse=True))[:10]
    
    print('Топ-10 всех слов', top_words)
    print('Топ-10 всех существительных', top_nouns)

### Что Тося говорит

In [51]:
WordsCounting('TosiaSays.txt')

Распределение по частям речи  Counter({'NOUN': 2530, 'NPRO': 1327, 'VERB': 1259, 'PRCL': 1234, 'CONJ': 559, 'ADVB': 523, 'ADJF': 475, 'PRED': 464, 'INTJ': 343, 'PREP': 294, 'INFN': 209, 'NUMR': 75, 'ADJS': 57, 'GRND': 40, 'COMP': 27, 'PRTS': 7})
Всего слов в этой части коруса (вместе с неопределенной частью речи) 21788
Топ-10 всех слов [('xочить', 102), ('няня', 100), ('ешчё', 88), ('ама', 83), ('смотреть', 77), ('видеть', 76), ('мочь', 62), ('большой', 62), ('варя', 54), ('тося', 52)]
Топ-10 всех существительных [('няня', 100), ('ешчё', 88), ('ама', 83), ('варя', 54), ('тося', 52), ('мама', 36), ('стасик', 33), ('метр', 25), ('папа', 24), ('цхилд', 21)]


### Что Тося слышит

In [52]:
WordsCounting('TosiaHears.txt')

Распределение по частям речи  Counter({'NOUN': 16218, 'VERB': 4498, 'CONJ': 3016, 'NPRO': 2884, 'PRCL': 2788, 'ADVB': 1942, 'PREP': 1725, 'ADJF': 1632, 'INFN': 902, 'INTJ': 662, 'PRED': 431, 'ADJS': 402, 'GRND': 223, 'NUMR': 168, 'COMP': 93, 'PRTS': 39, 'PRTF': 16})
Всего слов в этой части коруса (вместе с неопределенной частью речи) 48988
Топ-10 всех слов [('нанна', 567), ('тося', 454), ('давать', 301), ('смотреть', 163), ('ешчё', 161), ('мочь', 139), ('сказать', 135), ('весь', 135), ('тосй', 109), ('пойти', 108)]
Топ-10 всех существительных [('нанна', 567), ('тося', 454), ('ешчё', 161), ('тосй', 109), ('мама', 102), ('тоня', 63), ('варя', 62), ('свет', 59), ('молодец', 54), ('ромашка', 48)]


# Яша

## Скачивание файлов

In [53]:
folder = []
for i in os.walk('childes_Yasha'):
    folder.append(i)

Выводим список всех файлов и папок

In [26]:
#folder

In [54]:
filenames = []
for address, dirs, files in folder:
    for file in files:
        filenames.append(file)

Выводим список всех файлов, содержащих диалоги

In [19]:
#filenames

In [55]:
new = []
for filename in filenames:
    if filename[0].isnumeric():
        new.append(filename)
print(new)
print(len(new))

['10.txt', '11.txt', '12.txt', '13.06.2018.txt', '13.txt', '13_02_2018.txt', '14.txt', '15.txt', '16.txt', '18.txt', '19.txt', '20.txt', '21.cha', '22.cha', '22_02_2018.cha', '23.cha', '24.cha', '24_03_2018.cha', '25.cha', '26.cha', '27.01.2018.cha', '27.cha', '6_02_2018.txt', '7.txt', '8.txt']
25


In [56]:
paths = []  # список готовых директорий к файлам с диалогами
for filename in new:
    directory = 'childes_Yasha/'
    path = directory + filename
    paths.append(path)
    #print(path)

#print(len(paths))

## Разделение реплик на то, что ребенок слышит, и то, что он говорит

In [57]:
morph = pymorphy2.MorphAnalyzer()

dict_words = []

for filename in paths:
    with open(filename, 'r', encoding='utf-8') as f:
        file = f.readlines()
        textChild = ''
        textParents = ''
        for line in file:
            line = translit(line, 'ru')
            line = re.sub('йо', 'ё', line)
            if line.startswith('*ЦХИ'):  # выбираем только реплики ребенка 
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textChild += line.lower()
            elif line.startswith('*'):
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textParents += line.lower()
        
with open('YashaSays.txt', 'w+', encoding='utf-8') as f:
    f.write(textChild)
with open('YashaHears.txt', 'w+', encoding='utf-8') as f:
    f.write(textParents)

### Что Яша говорит

In [58]:
WordsCounting('YashaSays.txt')

Распределение по частям речи  Counter({'NOUN': 183, 'PREP': 53, 'INTJ': 33, 'VERB': 24, 'ADVB': 16, 'PRCL': 7, 'ADJF': 6, 'PRED': 5, 'NUMR': 4, 'CONJ': 3, 'INFN': 3, 'ADJS': 2})
Всего слов в этой части коруса (вместе с неопределенной частью речи) 1421
Топ-10 всех слов [('ала', 18), ('картошка', 15), ('деу', 14), ('мультик', 12), ('вилка', 10), ('дать', 10), ('пац', 7), ('алло', 7), ('спать', 6), ('мау', 6)]
Топ-10 всех существительных [('ала', 18), ('картошка', 15), ('мультик', 12), ('вилка', 10), ('яша', 5), ('папа', 4), ('миша', 4), ('пакетик', 4), ('мусор', 3), ('клякляу', 3)]


### Что Яша слышит

In [59]:
WordsCounting('YashaHears.txt')

Распределение по частям речи  Counter({'NOUN': 1274, 'CONJ': 472, 'VERB': 452, 'NPRO': 300, 'PRCL': 277, 'ADVB': 155, 'PREP': 127, 'ADJF': 81, 'INFN': 63, 'INTJ': 60, 'PRED': 38, 'ADJS': 17, 'COMP': 8, 'NUMR': 6, 'GRND': 4, 'PRTS': 2, 'PRTF': 1})
Всего слов в этой части коруса (вместе с неопределенной частью речи) 4813
Топ-10 всех слов [('яша', 109), ('написать', 37), ('говорить', 30), ('давать', 28), ('сказать', 24), ('спать', 24), ('ешчё', 20), ('пожалуйста', 19), ('картошка', 18), ('мусор', 17)]
Топ-10 всех существительных [('яша', 109), ('ешчё', 20), ('картошка', 18), ('мусор', 17), ('стол', 16), ('пупок', 16), ('салат', 12), ('буква', 12), ('коляска', 11), ('крокодил', 11)]
